In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import string


import re
# Base URL with a placeholder for the letter
base_url = "https://crs.upd.edu.ph/schedule/120241/{}"

all_rows = []
headers = None

for letter in string.ascii_uppercase:
    url = base_url.format(letter)
    print(f"Fetching data for letter: {letter} from URL: {url}")

    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve data for {letter}: HTTP {response.status_code}")
        continue

    soup = BeautifulSoup(response.text, "html.parser")

    table = soup.find("table", id="tbl_schedule")
    if not table:
        print(f"No table found for letter {letter}.")
        continue

    if headers is None:
        header_row = table.find("thead").find("tr")
        headers = [th.get_text(separator=" ", strip=True) for th in header_row.find_all("th")]
        print("Table headers:", headers)

    for tbody in table.find_all("tbody"):
        for row in tbody.find_all("tr"):
            cells = row.find_all("td")
            cell_texts = [cell.get_text(separator=" ", strip=True) for cell in cells]
            # In case the number of cells doesn't match header count, store raw list
            if len(cell_texts) == len(headers):
                row_data = dict(zip(headers, cell_texts))
            else:
                row_data = {"Raw Data": cell_texts}
            # Optionally add the letter the row came from
            row_data["Letter"] = letter
            all_rows.append(row_data)

df = pd.DataFrame(all_rows)
print("\nCombined DataFrame (first 5 rows):")
print(df.head())

Fetching data for letter: A from URL: https://crs.upd.edu.ph/schedule/120241/A
Table headers: ['Class Code', 'Class', 'Credits', 'Schedule Instructor(s) Remarks', 'Enlisting Unit : Block Block Remarks', 'Available Slots / Total Slots', 'Demand', 'Restrictions']
Fetching data for letter: B from URL: https://crs.upd.edu.ph/schedule/120241/B
Fetching data for letter: C from URL: https://crs.upd.edu.ph/schedule/120241/C
Fetching data for letter: D from URL: https://crs.upd.edu.ph/schedule/120241/D
Fetching data for letter: E from URL: https://crs.upd.edu.ph/schedule/120241/E
Fetching data for letter: F from URL: https://crs.upd.edu.ph/schedule/120241/F
Fetching data for letter: G from URL: https://crs.upd.edu.ph/schedule/120241/G
Fetching data for letter: H from URL: https://crs.upd.edu.ph/schedule/120241/H
Fetching data for letter: I from URL: https://crs.upd.edu.ph/schedule/120241/I
Fetching data for letter: J from URL: https://crs.upd.edu.ph/schedule/120241/J
Fetching data for letter: K

In [ ]:
df = df.drop(columns=["Letter", "Class Code", "Raw Data"])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

output_csv = "/content/drive/My Drive/crs_schedule_A_to_Z.csv"

# Save DataFrame as CSV directly to Drive
df.to_csv(output_csv, index=False)
print(f"\nData saved to '{output_csv}'.")

Mounted at /content/drive

Data saved to '/content/drive/My Drive/crs_schedule_A_to_Z.csv'.


In [ ]:
import re

In [ ]:
df.columns

Index(['Class', 'Credits', 'Schedule Instructor(s) Remarks',
       'Enlisting Unit : Block Block Remarks', 'Available Slots / Total Slots',
       'Demand', 'Restrictions'],
      dtype='object')

In [ ]:
df["Class"]

,Class
0,AI 201 HZZQ
1,AI 201 TZZQ
2,AI 211 MZZQ
3,AI 211 WFZ
4,AI 221 TZZQ
...,...
7780,WD 300 WD 300-1
7781,WD 300 WD 300-2
7782,NaN
7783,NaN


In [ ]:
import pandas as pd
import re

def add_space(time_str):
    """
    Converts "5:30PM" to "5:30 PM".
    """
    return re.sub(r'(\d)([APMapm]{2})$', r'\1 \2', time_str)

def parse_schedule(entry):
    """
    Parse an entry from the 'Schedule Instructor(s) Remarks' column.

    Expected example formats:
      "WF 5:30-7PM lec AECH-Seminar Room NAVAL, PROSPERO ..."
      "Th 6-9PM lec AECH-ERDT Rm NAVAL, PROSPERO ..."

    This function extracts:
      - days: e.g., "WF" or "Th"
      - start_time: e.g., "5:30 PM" (if missing its AM/PM, it inherits the end time's indicator)
      - end_time: e.g., "7 PM" (ensuring a space before the AM/PM part)
      - mode: e.g., "lec"
      - room: e.g., "AECH-Seminar Room"
          (captured greedily using .* up until the first comma)

    It uses a lookahead to stop capturing the room when a capitalized word followed by a comma (assumed instructor's name) is encountered.
    """
    pattern = re.compile(
        r'^(?P<days>(?:Th|M|T|W|F|S)+)\s+'
        r'(?P<start>\d{1,2}(?::\d{2})?)\s*(?P<start_ampm>[APMapm]{2})?-'
        r'(?P<end>\d{1,2}(?::\d{2})?\s*[APMapm]{2})\s+'
        r'(?P<mode>\w+)\s+'
        r'(?P<room>.*?)(?=\s+[A-Z][a-zA-Z]+,|$)',
        re.IGNORECASE
    )
    m = pattern.match(str(entry))
    if m:
        days = m.group("days").strip()
        start = m.group("start").strip()
        start_ampm = m.group("start_ampm")  # May be None
        end = m.group("end").strip()
        mode = m.group("mode").strip().lower()
        room = m.group("room").strip()

        # If the room field contains a comma, only keep the text before the comma.
        if ',' in room:
            room = room.split(",")[0].strip()

        # If room contains "TBA" anywhere, optionally just force it to be "TBA"
        if re.search(r'\bTBA\b', room, re.IGNORECASE):
            room = "TBA"

        # If start time is missing its AM/PM marker, infer it from the end time.
        if not start_ampm:
            end_ampm_match = re.search(r'([APMapm]{2})$', end)
            if end_ampm_match:
                start_ampm = end_ampm_match.group(1)
        start_time = add_space(start + (start_ampm if start_ampm else ""))
        end_time = add_space(end)
        return pd.Series([days, start_time, end_time, mode, room])
    else:
        return pd.Series([None, None, None, None, None])

# Assume the DataFrame is already loaded into df and the target column is exactly as below.
df[['days', 'start_time', 'end_time', 'mode', 'room']] = df["Schedule Instructor(s) Remarks"].apply(parse_schedule)

# Display the processed results
display(df[['Schedule Instructor(s) Remarks', 'days', 'start_time', 'end_time', 'mode', 'room']].head())


,Schedule Instructor(s) Remarks,days,start_time,end_time,mode,room
0,"Th 6-9PM lec AECH-ERDT Rm NAVAL, PROSPERO Face...",Th,6 PM,9 PM,lec,AECH-ERDT Rm
1,"T 6-9PM lec AECH-ERDT Rm NAVAL, PROSPERO Face-...",T,6 PM,9 PM,lec,AECH-ERDT Rm
2,"M 6-9PM lec TBA DALISAY, JON DEWITT Hybrid: A ...",M,6 PM,9 PM,lec,TBA
3,"WF 5:30-7PM lec AECH-Seminar Room REGONIA, PAU...",WF,5:30 PM,7 PM,lec,AECH-Seminar Room
4,"T 6-9PM lec TBA REMOLONA, MIGUEL FRANCISCO; DA...",T,6 PM,9 PM,lec,TBA


In [ ]:
import re
import pandas as pd

def compute_student_count(cell):
    """
    Processes a cell from the "Available Slots / Total Slots" column and calculates:
      Students Count = Total Slots - Available Slots.

    For entries like:
      "0 / 21"  → returns 21 - 0 = 21
      "OVERBOOKED / 20" → sets available slots to 0, returns 20 - 0 = 20

    If the cell is not a string or in an unexpected format, returns 0.
    """
    # Ensure the cell is a string.
    if not isinstance(cell, str):
        return 0

    # Replace newlines and tabs with spaces and trim extra whitespace.
    cell_clean = cell.replace('\n', ' ').replace('\t', ' ').strip()

    # Split the cell text by "/"
    parts = cell_clean.split('/')
    if len(parts) < 2:
        return 0

    available_text = parts[0].strip()
    total_text = parts[1].strip()

    try:
        # If available_text indicates "OVERBOOKED", then set available slots to 0.
        if "OVERBOOKED" in available_text.upper():
            available = 0
        else:
            available = int(re.sub(r'\D', '', available_text))
        total = int(re.sub(r'\D', '', total_text))
        return total - available
    except Exception as e:
        return 0

# Create a new column in the DataFrame for the computed student count.
df["Students Count"] = df["Available Slots / Total Slots"].apply(compute_student_count)

# Display the first few rows to verify.
display(df[["Available Slots / Total Slots", "Students Count"]].head())


,Available Slots / Total Slots,Students Count
0,0 /\n\t\t\t\t21,21
1,2 /\n\t\t\t\t20,18
2,OVERBOOKED /\n\t\t\t\t20,20
3,6 /\n\t\t\t\t20,14
4,0 /\n\t\t\t\t14,14


In [ ]:
# This will replace the "Class" column with just the first THREE words.
df["Class"] = df["Class"].apply(lambda x: ' '.join(str(x).split()[:4]))

# Display the first few rows to verify the change.
display(df.head())


,Class,Credits,Schedule Instructor(s) Remarks,Enlisting Unit : Block Block Remarks,Available Slots / Total Slots,Demand,Restrictions,days,start_time,end_time,mode,room,Students Count
0,AI 201 HZZQ,3.0,"Th 6-9PM lec AECH-ERDT Rm NAVAL, PROSPERO Face...",AIP,0 /\n\t\t\t\t21,0,For: AIP,Th,6 PM,9 PM,lec,AECH-ERDT Rm,21
1,AI 201 TZZQ,3.0,"T 6-9PM lec AECH-ERDT Rm NAVAL, PROSPERO Face-...",AIP,2 /\n\t\t\t\t20,0,For: AIP,T,6 PM,9 PM,lec,AECH-ERDT Rm,18
2,AI 211 MZZQ,3.0,"M 6-9PM lec TBA DALISAY, JON DEWITT Hybrid: A ...",AIP,OVERBOOKED /\n\t\t\t\t20,0,For: AIP,M,6 PM,9 PM,lec,TBA,20
3,AI 211 WFZ,3.0,"WF 5:30-7PM lec AECH-Seminar Room REGONIA, PAU...",AIP,6 /\n\t\t\t\t20,0,For: AIP,WF,5:30 PM,7 PM,lec,AECH-Seminar Room,14
4,AI 221 TZZQ,3.0,"T 6-9PM lec TBA REMOLONA, MIGUEL FRANCISCO; DA...",AIP,0 /\n\t\t\t\t14,0,,T,6 PM,9 PM,lec,TBA,14


In [ ]:
df

,Class,Credits,Schedule Instructor(s) Remarks,Enlisting Unit : Block Block Remarks,Available Slots / Total Slots,Demand,Restrictions,days,start_time,end_time,mode,room,Students Count
0,AI 201 HZZQ,3.0,"Th 6-9PM lec AECH-ERDT Rm NAVAL, PROSPERO Face...",AIP,0 /\n\t\t\t\t21,0,For: AIP,Th,6 PM,9 PM,lec,AECH-ERDT Rm,21
1,AI 201 TZZQ,3.0,"T 6-9PM lec AECH-ERDT Rm NAVAL, PROSPERO Face-...",AIP,2 /\n\t\t\t\t20,0,For: AIP,T,6 PM,9 PM,lec,AECH-ERDT Rm,18
2,AI 211 MZZQ,3.0,"M 6-9PM lec TBA DALISAY, JON DEWITT Hybrid: A ...",AIP,OVERBOOKED /\n\t\t\t\t20,0,For: AIP,M,6 PM,9 PM,lec,TBA,20
3,AI 211 WFZ,3.0,"WF 5:30-7PM lec AECH-Seminar Room REGONIA, PAU...",AIP,6 /\n\t\t\t\t20,0,For: AIP,WF,5:30 PM,7 PM,lec,AECH-Seminar Room,14
4,AI 221 TZZQ,3.0,"T 6-9PM lec TBA REMOLONA, MIGUEL FRANCISCO; DA...",AIP,0 /\n\t\t\t\t14,0,,T,6 PM,9 PM,lec,TBA,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7780,WD 300 WD 300-1,3.0,TBA TBA synchronous online and in person sessi...,WD,4 /\n\t\t\t\t10,0,,None,None,None,None,None,6
7781,WD 300 WD 300-2,3.0,TBA TBA synchronous online and in person sessi...,WD,6 /\n\t\t\t\t10,0,,None,None,None,None,None,4
7782,nan,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,0
7783,nan,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,0


In [ ]:
df.columns

Index(['Class', 'Credits', 'Schedule Instructor(s) Remarks',
       'Enlisting Unit : Block Block Remarks', 'Available Slots / Total Slots',
       'Demand', 'Restrictions', 'days', 'start_time', 'end_time', 'mode',
       'room', 'Students Count'],
      dtype='object')

In [ ]:
# Define the list of columns you want to keep
final_columns = ['Class', 'days', 'start_time', 'end_time', 'mode', 'room', 'Students Count']

# Create a new DataFrame that only contains these columns
df_final = df[final_columns]

# Display a preview of the cleaned data
display(df_final.head())


,Class,days,start_time,end_time,mode,room,Students Count
0,AI 201 HZZQ,Th,6 PM,9 PM,lec,AECH-ERDT Rm,21
1,AI 201 TZZQ,T,6 PM,9 PM,lec,AECH-ERDT Rm,18
2,AI 211 MZZQ,M,6 PM,9 PM,lec,TBA,20
3,AI 211 WFZ,WF,5:30 PM,7 PM,lec,AECH-Seminar Room,14
4,AI 221 TZZQ,T,6 PM,9 PM,lec,TBA,14


In [ ]:
df_final

,Class,days,start_time,end_time,mode,room,Students Count
0,AI 201 HZZQ,Th,6 PM,9 PM,lec,AECH-ERDT Rm,21
1,AI 201 TZZQ,T,6 PM,9 PM,lec,AECH-ERDT Rm,18
2,AI 211 MZZQ,M,6 PM,9 PM,lec,TBA,20
3,AI 211 WFZ,WF,5:30 PM,7 PM,lec,AECH-Seminar Room,14
4,AI 221 TZZQ,T,6 PM,9 PM,lec,TBA,14
...,...,...,...,...,...,...,...
7780,WD 300 WD 300-1,None,None,None,None,None,6
7781,WD 300 WD 300-2,None,None,None,None,None,4
7782,nan,None,None,None,None,None,0
7783,nan,None,None,None,None,None,0


In [ ]:
df_final.rename(columns={
    'Class': 'class',
    'Students Count': 'count'
}, inplace=True)


<ipython-input-14-f36f867fae31>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.rename(columns={


In [ ]:
df_final

,class,days,start_time,end_time,mode,room,count
0,AI 201 HZZQ,Th,6 PM,9 PM,lec,AECH-ERDT Rm,21
1,AI 201 TZZQ,T,6 PM,9 PM,lec,AECH-ERDT Rm,18
2,AI 211 MZZQ,M,6 PM,9 PM,lec,TBA,20
3,AI 211 WFZ,WF,5:30 PM,7 PM,lec,AECH-Seminar Room,14
4,AI 221 TZZQ,T,6 PM,9 PM,lec,TBA,14
...,...,...,...,...,...,...,...
7780,WD 300 WD 300-1,None,None,None,None,None,6
7781,WD 300 WD 300-2,None,None,None,None,None,4
7782,nan,None,None,None,None,None,0
7783,nan,None,None,None,None,None,0


In [ ]:
df_final.dropna(subset=['days', 'start_time', 'end_time', 'mode', 'room'], inplace=True)
df_final.reset_index(drop=True, inplace=True)


<ipython-input-16-8f005eda1d23>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.dropna(subset=['days', 'start_time', 'end_time', 'mode', 'room'], inplace=True)


In [ ]:
df_final

,class,days,start_time,end_time,mode,room,count
0,AI 201 HZZQ,Th,6 PM,9 PM,lec,AECH-ERDT Rm,21
1,AI 201 TZZQ,T,6 PM,9 PM,lec,AECH-ERDT Rm,18
2,AI 211 MZZQ,M,6 PM,9 PM,lec,TBA,20
3,AI 211 WFZ,WF,5:30 PM,7 PM,lec,AECH-Seminar Room,14
4,AI 221 TZZQ,T,6 PM,9 PM,lec,TBA,14
...,...,...,...,...,...,...,...
6549,WD 231 FZQ,F,5:30 PM,8:30 PM,lec,TBA,11
6550,"WD 270 MZQ Gender,",M,5:30 PM,8:30 PM,lec,TBA,10
6551,WD 271 WZQ Gender,W,5:30 PM,8:30 PM,lec,TBA,10
6552,WD 291 TZQ,T,5:30 PM,8:30 PM,disc,TBA,8


In [ ]:
df = df_final

In [ ]:
# Define targets for each group
time_targets = {
    'TTh 11:30 AM': (df['days'] == 'TTh') & (df['end_time'] == '11:30 AM'),
    'WF 11:30 AM': (df['days'] == 'WF') & (df['end_time'] == '11:30 AM'),
    'TTh 4 PM': (df['days'] == 'TTh') & (df['end_time'] == '4 PM'),
    'WF 4 PM': (df['days'] == 'WF') & (df['end_time'] == '4 PM'),
}

# Container for results
results = []

# Aggregate by room for each target
for label, condition in time_targets.items():
    subset = df.loc[condition]
    agg = subset.groupby('room')['count'].sum().reset_index()
    agg['slot'] = label
    results.append(agg)

# Combine all into one DataFrame and sort
final_room_counts = pd.concat(results).sort_values(by=['slot', 'count'], ascending=[True, False])



In [ ]:
final_room_counts

,room,count,slot
223,TBA,1830,TTh 11:30 AM
66,CS AUDI,100,TTh 11:30 AM
71,EEEI 120,75,TTh 11:30 AM
47,CAL 312 CHUA-MANANSALA,60,TTh 11:30 AM
174,PH 116-118,60,TTh 11:30 AM
...,...,...,...
149,PH 400B,10,WF 4 PM
4,AH 206-208,7,WF 4 PM
109,NIGS 127 DI,7,WF 4 PM
118,NIP R211,6,WF 4 PM


In [ ]:
final_room_counts.to_csv("final_room_counts.csv", index=False)